In [1]:
!pip install spotipy

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="426e22a5ddc84900acb6d605bb0a2cdf",
                                                           client_secret="a49cdbf32d6c46beb53c014afad4078f"))

### Defining the functions to obtain 1) songs & artists, and 2) audio features 

In [3]:
def get_id_song_artist(username, playlist_id):
    # Getting all tracks in the playlist
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']

    # Continuing getting tracks until there are no more tracks to get
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    # Looping through each track and extract ID, song name, and artist name
    id_song_artist_list = []
    for track in tracks:
        track_info = track['track']
        if 'id' in track_info:
            track_id = track_info['id']
            track_name = track_info['name']
            artist_name = track_info['artists'][0]['name']
            id_song_artist_list.append({'id': track_id, 'name': track_name, 'artist': artist_name})

    # Converting the list of track IDs, song names, and artist names to a pandas dataframe and returning it
    return pd.DataFrame(id_song_artist_list)

In [4]:
def get_playlist_audio_features(playlist_id):
    # Function to get audio features for a list of track IDs
    def get_audio_features(track_ids):
        # Filtering out None values from track_ids list
        track_ids = [id for id in track_ids if id is not None]

        # Getting audio features for the track IDs
        features = sp.audio_features(tracks=track_ids)
        return features

    # Getting all tracks in the playlist
    audio_features = []

    # Getting the first batch of tracks
    results = sp.playlist_items(playlist_id, limit=100)

    # Continuing getting tracks in batches of 100 until there are no more tracks to get
    while results['items']:
        audio_features.extend(results['items'])
        results = sp.playlist_items(playlist_id, offset=len(audio_features), limit=100)

    # Splitting the playlist into batches of 100 tracks
    batches = [audio_features[i:i+100] for i in range(0, len(audio_features), 100)]

    # Looping through each batch and get audio features
    features_list = []
    for batch in batches:
        # Extracting track IDs from batch
        track_ids = [track['track']['id'] for track in batch if track['track'] is not None]

        # Getting audio features for the track IDs
        features = get_audio_features(track_ids)

        # Appending the features to the list
        features_list.extend(features)

    # Converting the list of features to a pandas dataframe and returining it
    return pd.DataFrame(features_list)

#### Obtaining a collection of songs & artists, and audio features from a long Spotify playlist with many different types of genres

In [5]:
id_song_artist_df1 = get_id_song_artist("spotify", "6yPiKpy7evrwvZodByKvM9")
audio_features_df1 = get_playlist_audio_features("6yPiKpy7evrwvZodByKvM9")

merged_df1 = pd.merge(id_song_artist_df1, audio_features_df1, on='id', how='inner')
merged_df1.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df1.dropna(inplace=True)
merged_df1

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,33xMbeHzmWd6Od0BmLZEUs,2K,Nosaj Thing,0.310,0.445,7,-13.355,0,0.0863,0.09400,0.067800,0.1130,0.122,95.360,audio_features,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560,3
1,3UnyplmZaq547hwsfOR5yy,4 Billion Souls,The Doors,0.419,0.565,5,-11.565,1,0.0347,0.13700,0.337000,0.1280,0.648,151.277,audio_features,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707,4
2,1w8QCSDH4QobcQeT4uMKLm,4 Minute Warning,Radiohead,0.354,0.302,9,-13.078,1,0.0326,0.59000,0.070900,0.1110,0.223,123.753,audio_features,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285,4
3,7J9mBHG4J2eIfDAv5BehKA,7 Element,Vitas,0.727,0.785,5,-6.707,0,0.0603,0.32500,0.126000,0.3100,0.960,129.649,audio_features,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940,4
4,1VZedwJj1gyi88WFRhfThb,#9 Dream,R.E.M.,0.571,0.724,0,-5.967,1,0.0260,0.02310,0.003110,0.0919,0.385,116.755,audio_features,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10002,07CyrZF9eVd02zzIse7tZA,In Between Days - 2006 Remaster,The Cure,0.541,0.830,9,-6.246,1,0.0316,0.10900,0.701000,0.1670,0.711,142.592,audio_features,spotify:track:07CyrZF9eVd02zzIse7tZA,https://api.spotify.com/v1/tracks/07CyrZF9eVd0...,https://api.spotify.com/v1/audio-analysis/07Cy...,177987,4
10003,5jzma6gCzYtKB1DbEwFZKH,London Calling - Remastered,The Clash,0.651,0.801,0,-7.340,1,0.0514,0.12300,0.000000,0.0825,0.776,133.769,audio_features,spotify:track:5jzma6gCzYtKB1DbEwFZKH,https://api.spotify.com/v1/tracks/5jzma6gCzYtK...,https://api.spotify.com/v1/audio-analysis/5jzm...,200480,4
10005,60rIdEPDrzyLiLC0icp3xz,Flower,Moby,0.686,0.610,7,-5.902,1,0.0262,0.19000,0.000931,0.0710,0.766,80.567,audio_features,spotify:track:60rIdEPDrzyLiLC0icp3xz,https://api.spotify.com/v1/tracks/60rIdEPDrzyL...,https://api.spotify.com/v1/audio-analysis/60rI...,206293,4
10006,0sEm1ld0V8YTCPcjPVfIsc,Brighter Than Gold,The Cat Empire,0.711,0.718,6,-5.739,1,0.0380,0.00824,0.002080,0.0732,0.688,117.071,audio_features,spotify:track:0sEm1ld0V8YTCPcjPVfIsc,https://api.spotify.com/v1/tracks/0sEm1ld0V8YT...,https://api.spotify.com/v1/audio-analysis/0sEm...,200293,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Pop playlist

In [6]:
id_song_artist_df2 = get_id_song_artist("spotify", "37i9dQZF1DX1ngEVM0lKrb")
audio_features_df2 = get_playlist_audio_features("37i9dQZF1DX1ngEVM0lKrb")

merged_df2 = pd.merge(id_song_artist_df2, audio_features_df2, on='id', how='inner')
merged_df2.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df2.dropna(inplace=True)
merged_df2

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,0yLdNVWF3Srea0uzk55zFn,Flowers,Miley Cyrus,0.707,0.681,0,-4.325,1,0.0668,0.06320,0.000005,0.0322,0.646,117.999,audio_features,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4
1,4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),David Guetta,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,audio_features,spotify:track:4uUG5RXrOk84mYEfFvj3cK,https://api.spotify.com/v1/tracks/4uUG5RXrOk84...,https://api.spotify.com/v1/audio-analysis/4uUG...,175238,4
2,0WtM2NBVQNNJLh6scP13H8,Calm Down (with Selena Gomez),Rema,0.801,0.806,11,-5.206,1,0.0381,0.38200,0.000669,0.1140,0.802,106.999,audio_features,spotify:track:0WtM2NBVQNNJLh6scP13H8,https://api.spotify.com/v1/tracks/0WtM2NBVQNNJ...,https://api.spotify.com/v1/audio-analysis/0WtM...,239318,4
3,0V3wPSX9ygBnCm8psDIegu,Anti-Hero,Taylor Swift,0.637,0.643,4,-6.571,1,0.0519,0.13000,0.000002,0.1420,0.533,97.008,audio_features,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
4,3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,audio_features,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1I8tHoNBFTuoJAlh4hfVVE,Firestone,Kygo,0.704,0.634,11,-7.374,0,0.0428,0.39300,0.000032,0.0952,0.411,113.927,audio_features,spotify:track:1I8tHoNBFTuoJAlh4hfVVE,https://api.spotify.com/v1/tracks/1I8tHoNBFTuo...,https://api.spotify.com/v1/audio-analysis/1I8t...,271640,4
146,4wCmqSrbyCgxEXROQE6vtV,Somebody That I Used To Know,Gotye,0.864,0.495,0,-7.036,1,0.0370,0.59100,0.000133,0.0992,0.720,129.062,audio_features,spotify:track:4wCmqSrbyCgxEXROQE6vtV,https://api.spotify.com/v1/tracks/4wCmqSrbyCgx...,https://api.spotify.com/v1/audio-analysis/4wCm...,244973,4
147,62aP9fBQKYKxi7PDXwcUAS,ily (i love you baby) (feat. Emilee),Surf Mesa,0.674,0.774,11,-7.567,0,0.0892,0.06860,0.001880,0.3930,0.330,112.050,audio_features,spotify:track:62aP9fBQKYKxi7PDXwcUAS,https://api.spotify.com/v1/tracks/62aP9fBQKYKx...,https://api.spotify.com/v1/audio-analysis/62aP...,176547,4
148,1KxwZYyzWNyZSRyErj2ojT,Let Her Go,Passenger,0.479,0.545,7,-7.346,1,0.0688,0.36500,0.000000,0.0963,0.243,74.897,audio_features,spotify:track:1KxwZYyzWNyZSRyErj2ojT,https://api.spotify.com/v1/tracks/1KxwZYyzWNyZ...,https://api.spotify.com/v1/audio-analysis/1Kxw...,252733,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Rock playlist

In [7]:
id_song_artist_df3 = get_id_song_artist("spotify", "37i9dQZF1DWXRqgorJj26U")
audio_features_df3 = get_playlist_audio_features("37i9dQZF1DWXRqgorJj26U")

merged_df3 = pd.merge(id_song_artist_df3, audio_features_df3, on='id', how='inner')
merged_df3.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df3.dropna(inplace=True)
merged_df3

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5e9TFTbltYBg2xThimr0rU,The Chain - 2004 Remaster,Fleetwood Mac,0.545,0.670,9,-8.810,1,0.0496,0.009000,0.000822,0.0451,0.481,151.553,audio_features,spotify:track:5e9TFTbltYBg2xThimr0rU,https://api.spotify.com/v1/tracks/5e9TFTbltYBg...,https://api.spotify.com/v1/audio-analysis/5e9T...,270213,4
1,40riOy7x9W7GXjyGp4pjAv,Hotel California - 2013 Remaster,Eagles,0.579,0.508,2,-9.484,1,0.0270,0.005740,0.000494,0.0575,0.609,147.125,audio_features,spotify:track:40riOy7x9W7GXjyGp4pjAv,https://api.spotify.com/v1/tracks/40riOy7x9W7G...,https://api.spotify.com/v1/audio-analysis/40ri...,391376,4
2,57JVGBtBLCfHw2muk5416J,Another One Bites The Dust - Remastered 2011,Queen,0.933,0.528,5,-6.472,0,0.1620,0.112000,0.329000,0.1630,0.756,109.975,audio_features,spotify:track:57JVGBtBLCfHw2muk5416J,https://api.spotify.com/v1/tracks/57JVGBtBLCfH...,https://api.spotify.com/v1/audio-analysis/57JV...,214653,4
3,1gzIbdFnGJ226LTl0Cn2SX,White Wedding - Pt. 1,Billy Idol,0.673,0.814,9,-5.411,1,0.0367,0.013300,0.002930,0.3430,0.728,147.193,audio_features,spotify:track:1gzIbdFnGJ226LTl0Cn2SX,https://api.spotify.com/v1/tracks/1gzIbdFnGJ22...,https://api.spotify.com/v1/audio-analysis/1gzI...,252200,4
4,57bgtoPSgt236HzfBOd8kj,Thunderstruck,AC/DC,0.502,0.890,4,-5.175,1,0.0364,0.000147,0.011700,0.2170,0.259,133.520,audio_features,spotify:track:57bgtoPSgt236HzfBOd8kj,https://api.spotify.com/v1/tracks/57bgtoPSgt23...,https://api.spotify.com/v1/audio-analysis/57bg...,292880,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2aoo2jlRnM3A0NyLQqMN2f,All Along the Watchtower,Jimi Hendrix,0.438,0.805,8,-6.237,1,0.0624,0.002550,0.000057,0.0842,0.564,113.253,audio_features,spotify:track:2aoo2jlRnM3A0NyLQqMN2f,https://api.spotify.com/v1/tracks/2aoo2jlRnM3A...,https://api.spotify.com/v1/audio-analysis/2aoo...,240800,4
196,5o37x6LpNSec4jSfPIChsQ,We Got The Beat - 2011 Remaster,The Go-Go's,0.595,0.996,2,-3.418,1,0.0599,0.195000,0.034500,0.0351,0.376,152.049,audio_features,spotify:track:5o37x6LpNSec4jSfPIChsQ,https://api.spotify.com/v1/tracks/5o37x6LpNSec...,https://api.spotify.com/v1/audio-analysis/5o37...,151107,4
197,4hq0S6wznq7SHDyMOFXL9i,Tumbling Dice,The Rolling Stones,0.615,0.910,11,-4.248,1,0.0347,0.500000,0.511000,0.2810,0.770,111.013,audio_features,spotify:track:4hq0S6wznq7SHDyMOFXL9i,https://api.spotify.com/v1/tracks/4hq0S6wznq7S...,https://api.spotify.com/v1/audio-analysis/4hq0...,226333,4
198,6N0AnkdDFZUetw8KAGHV7e,Godzilla,Blue Öyster Cult,0.373,0.869,4,-7.719,1,0.0554,0.164000,0.000008,0.6050,0.696,184.024,audio_features,spotify:track:6N0AnkdDFZUetw8KAGHV7e,https://api.spotify.com/v1/tracks/6N0AnkdDFZUe...,https://api.spotify.com/v1/audio-analysis/6N0A...,221400,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Latin playlist

In [8]:
id_song_artist_df4 = get_id_song_artist("spotify", "37i9dQZF1DXbLMw3ry7d7k")
audio_features_df4 = get_playlist_audio_features("37i9dQZF1DXbLMw3ry7d7k")

merged_df4 = pd.merge(id_song_artist_df4, audio_features_df4, on='id', how='inner')
merged_df4.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df4.dropna(inplace=True)
merged_df4

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,6Sq7ltF9Qa7SNFBsV5Cogx,Me Porto Bonito,Bad Bunny,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,audio_features,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4
1,6TBzRwnX2oYd8aOrOuyK1p,Fin de Semana,Oscar Maydon,0.699,0.541,0,-4.943,1,0.0826,0.0628,0.000000,0.0916,0.365,98.395,audio_features,spotify:track:6TBzRwnX2oYd8aOrOuyK1p,https://api.spotify.com/v1/tracks/6TBzRwnX2oYd...,https://api.spotify.com/v1/audio-analysis/6TBz...,224921,4
2,6G12ZafqofSq7YtrMqUm76,Monotonía,Shakira,0.873,0.695,0,-4.207,0,0.0459,0.4190,0.000001,0.2110,0.823,131.963,audio_features,spotify:track:6G12ZafqofSq7YtrMqUm76,https://api.spotify.com/v1/tracks/6G12ZafqofSq...,https://api.spotify.com/v1/audio-analysis/6G12...,158805,4
3,23Lyy7ZXRvzfgH4JtDkKrX,No Se Va - En Vivo,Grupo Frontera,0.594,0.527,0,-9.335,1,0.0335,0.1210,0.000000,0.2340,0.692,172.712,audio_features,spotify:track:23Lyy7ZXRvzfgH4JtDkKrX,https://api.spotify.com/v1/tracks/23Lyy7ZXRvzf...,https://api.spotify.com/v1/audio-analysis/23Ly...,194019,4
4,7dSZ6zGTQx66c2GF91xCrb,PROVENZA,KAROL G,0.870,0.516,1,-8.006,1,0.0541,0.6560,0.008230,0.1100,0.530,111.005,audio_features,spotify:track:7dSZ6zGTQx66c2GF91xCrb,https://api.spotify.com/v1/tracks/7dSZ6zGTQx66...,https://api.spotify.com/v1/audio-analysis/7dSZ...,210200,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20ChN6ZWJ2yhuXcE6SKHDM,Wow Wow (feat. Becky G),Maria Becerra,0.744,0.679,5,-3.510,1,0.0791,0.0654,0.004290,0.0966,0.866,176.024,audio_features,spotify:track:20ChN6ZWJ2yhuXcE6SKHDM,https://api.spotify.com/v1/tracks/20ChN6ZWJ2yh...,https://api.spotify.com/v1/audio-analysis/20Ch...,199773,4
96,3M23T72K5eqHI2GzdFMMHE,Brother,Anuel AA,0.592,0.625,10,-6.203,0,0.2000,0.6180,0.000097,0.2870,0.555,166.018,audio_features,spotify:track:3M23T72K5eqHI2GzdFMMHE,https://api.spotify.com/v1/tracks/3M23T72K5eqH...,https://api.spotify.com/v1/audio-analysis/3M23...,209579,4
97,5cy2dREJqJAyf3CNr509W7,PENDEJO,Enrique Iglesias,0.890,0.666,7,-4.361,1,0.1290,0.0606,0.000065,0.1100,0.269,98.018,audio_features,spotify:track:5cy2dREJqJAyf3CNr509W7,https://api.spotify.com/v1/tracks/5cy2dREJqJAy...,https://api.spotify.com/v1/audio-analysis/5cy2...,165360,4
98,5b1je14Oz92lpIlIMLieWw,Viaje,Farruko,0.705,0.799,0,-4.776,0,0.0347,0.0357,0.417000,0.0643,0.658,122.018,audio_features,spotify:track:5b1je14Oz92lpIlIMLieWw,https://api.spotify.com/v1/tracks/5b1je14Oz92l...,https://api.spotify.com/v1/audio-analysis/5b1j...,303627,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Electronic/Dance playlist

In [9]:
id_song_artist_df5 = get_id_song_artist("spotify", "1dvoCOb3vso33rTd4FWqRW")
audio_features_df5 = get_playlist_audio_features("1dvoCOb3vso33rTd4FWqRW")

merged_df5 = pd.merge(id_song_artist_df5, audio_features_df5, on='id', how='inner')
merged_df5.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df5.dropna(inplace=True)
merged_df5

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2CgOd0Lj5MuvOqzqdaAXtS,Shelter,Porter Robinson,0.276,0.796,0,-4.499,1,0.0579,0.002930,0.000000,0.1840,0.260,199.526,audio_features,spotify:track:2CgOd0Lj5MuvOqzqdaAXtS,https://api.spotify.com/v1/tracks/2CgOd0Lj5Muv...,https://api.spotify.com/v1/audio-analysis/2CgO...,218965,4
1,2V65y3PX4DkRhy1djlxd9p,Don't You Worry Child - Radio Edit,Swedish House Mafia,0.612,0.840,11,-3.145,0,0.0509,0.112000,0.000000,0.1160,0.438,129.042,audio_features,spotify:track:2V65y3PX4DkRhy1djlxd9p,https://api.spotify.com/v1/tracks/2V65y3PX4DkR...,https://api.spotify.com/v1/audio-analysis/2V65...,212862,4
2,0JhKJg5ejeQ8jq89UQtnw8,Levels - Radio Edit,Avicii,0.620,0.881,1,-5.851,0,0.0372,0.058500,0.714000,0.3110,0.415,126.035,audio_features,spotify:track:0JhKJg5ejeQ8jq89UQtnw8,https://api.spotify.com/v1/tracks/0JhKJg5ejeQ8...,https://api.spotify.com/v1/audio-analysis/0JhK...,199904,4
3,1dFkD1JfRMzwO6hwUsE8aS,Spectrum,Zedd,0.609,0.801,0,-4.417,0,0.0340,0.013700,0.003490,0.1270,0.407,129.990,audio_features,spotify:track:1dFkD1JfRMzwO6hwUsE8aS,https://api.spotify.com/v1/tracks/1dFkD1JfRMzw...,https://api.spotify.com/v1/audio-analysis/1dFk...,243413,4
4,4Mq3WBbOdwmy4qDxV3sN5J,Pressure - Alesso UK Radio Edit,Nadia Ali,0.516,0.907,8,-6.412,1,0.0339,0.000118,0.400000,0.2920,0.115,128.046,audio_features,spotify:track:4Mq3WBbOdwmy4qDxV3sN5J,https://api.spotify.com/v1/tracks/4Mq3WBbOdwmy...,https://api.spotify.com/v1/audio-analysis/4Mq3...,163578,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,5yECq6R2S2iTV8GTxen4JV,Sleepless - Radio Edit,CAZZETTE,0.723,0.602,6,-10.573,0,0.0451,0.143000,0.000607,0.2990,0.561,124.006,audio_features,spotify:track:5yECq6R2S2iTV8GTxen4JV,https://api.spotify.com/v1/tracks/5yECq6R2S2iT...,https://api.spotify.com/v1/audio-analysis/5yEC...,221421,4
658,2us0RMthAhSWdRMGFEDITO,Liberate - Lane 8 Remix,Eric Prydz,0.713,0.578,0,-8.130,1,0.0416,0.024500,0.706000,0.0782,0.262,120.973,audio_features,spotify:track:2us0RMthAhSWdRMGFEDITO,https://api.spotify.com/v1/tracks/2us0RMthAhSW...,https://api.spotify.com/v1/audio-analysis/2us0...,314840,4
659,4FWzhh5B1rowe3qWKId1x3,blue - Jai Wolf Remix,keshi,0.464,0.448,0,-8.116,1,0.0266,0.090900,0.000008,0.1980,0.176,90.082,audio_features,spotify:track:4FWzhh5B1rowe3qWKId1x3,https://api.spotify.com/v1/tracks/4FWzhh5B1row...,https://api.spotify.com/v1/audio-analysis/4FWz...,202667,4
660,7mYp88qI0fTKLWr93vHTJM,Kingdom,ARTY,0.533,0.731,4,-3.224,1,0.0322,0.135000,0.000003,0.1980,0.312,127.980,audio_features,spotify:track:7mYp88qI0fTKLWr93vHTJM,https://api.spotify.com/v1/tracks/7mYp88qI0fTK...,https://api.spotify.com/v1/audio-analysis/7mYp...,240469,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Hip-Hop playlist

In [10]:
id_song_artist_df6 = get_id_song_artist("spotify", "6QY4S9npgtFDwd5egrtrGQ")
audio_features_df6 = get_playlist_audio_features("6QY4S9npgtFDwd5egrtrGQ")

merged_df6 = pd.merge(id_song_artist_df6, audio_features_df6, on='id', how='inner')
merged_df6.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df6.dropna(inplace=True)
merged_df6

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,4LujRMTMwAfFKjm45tixHY,Straight Outta Compton,N.W.A.,0.831,0.935,8,-6.020,1,0.1970,0.03740,0.000000,0.3300,0.367,102.836,audio_features,spotify:track:4LujRMTMwAfFKjm45tixHY,https://api.spotify.com/v1/tracks/4LujRMTMwAfF...,https://api.spotify.com/v1/audio-analysis/4Luj...,259227,4
1,771lQyKAtqXRpZ7iM4JsbP,Fuck Tha Police,N.W.A.,0.871,0.819,8,-5.439,0,0.2980,0.01380,0.000000,0.0608,0.857,98.699,audio_features,spotify:track:771lQyKAtqXRpZ7iM4JsbP,https://api.spotify.com/v1/tracks/771lQyKAtqXR...,https://api.spotify.com/v1/audio-analysis/771l...,345733,4
2,7L7JF5RZ2s5Pb0qSV3V5cW,No Vaseline,Ice Cube,0.574,0.927,11,-5.536,0,0.3770,0.03840,0.000000,0.1370,0.510,105.575,audio_features,spotify:track:7L7JF5RZ2s5Pb0qSV3V5cW,https://api.spotify.com/v1/tracks/7L7JF5RZ2s5P...,https://api.spotify.com/v1/audio-analysis/7L7J...,312467,4
3,5cV7iNvMXJyMhFNY0HFbQh,Gangsta Gangsta,N.W.A.,0.907,0.818,6,-3.777,0,0.2600,0.03850,0.000000,0.2930,0.721,98.093,audio_features,spotify:track:5cV7iNvMXJyMhFNY0HFbQh,https://api.spotify.com/v1/tracks/5cV7iNvMXJyM...,https://api.spotify.com/v1/audio-analysis/5cV7...,336707,4
4,3JdyhQORbJkCWT8xfLd8gD,Boyz-N-The-Hood - Remix,Eazy-E,0.837,0.692,7,-4.251,1,0.3370,0.03830,0.000000,0.2120,0.805,87.775,audio_features,spotify:track:3JdyhQORbJkCWT8xfLd8gD,https://api.spotify.com/v1/tracks/3JdyhQORbJkC...,https://api.spotify.com/v1/audio-analysis/3Jdy...,379733,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,62ESrMBSqn9D80w5mWEu5v,O.G. (feat. Nate Dogg),Snoop Dogg,0.511,0.903,2,-4.420,1,0.1420,0.00182,0.000004,0.1340,0.399,90.364,audio_features,spotify:track:62ESrMBSqn9D80w5mWEu5v,https://api.spotify.com/v1/tracks/62ESrMBSqn9D...,https://api.spotify.com/v1/audio-analysis/62ES...,272947,4
1293,1tXgkOfHwglherrInDcCWk,Doggystyle (feat. George Clinton & Jewell),Snoop Dogg,0.650,0.684,8,-5.261,1,0.0432,0.07830,0.000000,0.7640,0.552,99.971,audio_features,spotify:track:1tXgkOfHwglherrInDcCWk,https://api.spotify.com/v1/tracks/1tXgkOfHwglh...,https://api.spotify.com/v1/audio-analysis/1tXg...,334547,4
1296,46Lxn9dXh5u043tjYcGNfV,Murder Was The Case,Snoop Dogg,0.393,0.940,2,-4.254,1,0.0768,0.01180,0.000000,0.0622,0.323,176.618,audio_features,spotify:track:46Lxn9dXh5u043tjYcGNfV,https://api.spotify.com/v1/tracks/46Lxn9dXh5u0...,https://api.spotify.com/v1/audio-analysis/46Lx...,260853,4
1298,6M9BMY5Smb21xrvx34IMMj,21 Jumpstreet,Snoop Dogg,0.825,0.864,10,-4.387,0,0.2560,0.02860,0.000000,0.3080,0.731,95.819,audio_features,spotify:track:6M9BMY5Smb21xrvx34IMMj,https://api.spotify.com/v1/tracks/6M9BMY5Smb21...,https://api.spotify.com/v1/audio-analysis/6M9B...,325347,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Indie playlist

In [11]:
id_song_artist_df7 = get_id_song_artist("spotify", "37i9dQZF1DX26DKvjp0s9M")
audio_features_df7 = get_playlist_audio_features("37i9dQZF1DX26DKvjp0s9M")

merged_df7 = pd.merge(id_song_artist_df7, audio_features_df7, on='id', how='inner')
merged_df7.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df7.dropna(inplace=True)
merged_df7

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,The Neighbourhood,0.612,0.807,10,-2.810,1,0.0336,0.04950,0.017700,0.101,0.398,124.053,audio_features,spotify:track:2QjOHCTQ1Jl3zawyYOpxh6,https://api.spotify.com/v1/tracks/2QjOHCTQ1Jl3...,https://api.spotify.com/v1/audio-analysis/2QjO...,240400,4
1,2L9N0zZnd37dwF0clgxMGI,ceilings,Lizzy McAlpine,0.516,0.322,9,-11.762,1,0.0292,0.47300,0.001940,0.215,0.261,148.005,audio_features,spotify:track:2L9N0zZnd37dwF0clgxMGI,https://api.spotify.com/v1/tracks/2L9N0zZnd37d...,https://api.spotify.com/v1/audio-analysis/2L9N...,182888,3
2,4HfLQJtVT1KiX1eVedDyTm,Alaska,Maggie Rogers,0.847,0.386,9,-10.668,0,0.0507,0.41100,0.001760,0.108,0.178,104.001,audio_features,spotify:track:4HfLQJtVT1KiX1eVedDyTm,https://api.spotify.com/v1/tracks/4HfLQJtVT1Ki...,https://api.spotify.com/v1/audio-analysis/4HfL...,188000,4
3,5FVd6KXrgO9B3JPmC8OPst,Do I Wanna Know?,Arctic Monkeys,0.548,0.532,5,-7.596,1,0.0323,0.18600,0.000263,0.217,0.405,85.030,audio_features,spotify:track:5FVd6KXrgO9B3JPmC8OPst,https://api.spotify.com/v1/tracks/5FVd6KXrgO9B...,https://api.spotify.com/v1/audio-analysis/5FVd...,272394,4
4,4k6Uh1HXdhtusDW5y8Gbvy,Bad Habit,Steve Lacy,0.686,0.494,1,-7.093,1,0.0355,0.61300,0.000058,0.402,0.700,168.946,audio_features,spotify:track:4k6Uh1HXdhtusDW5y8Gbvy,https://api.spotify.com/v1/tracks/4k6Uh1HXdhtu...,https://api.spotify.com/v1/audio-analysis/4k6U...,232067,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4K3gM7kzX4POAfrs71HomN,Lookalike,Conan Gray,0.434,0.302,10,-10.117,1,0.0378,0.76600,0.000008,0.149,0.173,121.645,audio_features,spotify:track:4K3gM7kzX4POAfrs71HomN,https://api.spotify.com/v1/tracks/4K3gM7kzX4PO...,https://api.spotify.com/v1/audio-analysis/4K3g...,220867,4
96,55DyBUkjebkcYhDGfEXitR,Cariño,The Marías,0.647,0.385,0,-13.717,0,0.0295,0.40200,0.056900,0.117,0.660,113.982,audio_features,spotify:track:55DyBUkjebkcYhDGfEXitR,https://api.spotify.com/v1/tracks/55DyBUkjebkc...,https://api.spotify.com/v1/audio-analysis/55Dy...,258714,4
97,46RNrAkGsqWTDrv2ZPOAbx,Moon Song,Phoebe Bridgers,0.339,0.243,1,-12.512,1,0.0560,0.86900,0.003690,0.107,0.183,154.909,audio_features,spotify:track:46RNrAkGsqWTDrv2ZPOAbx,https://api.spotify.com/v1/tracks/46RNrAkGsqWT...,https://api.spotify.com/v1/audio-analysis/46RN...,277677,3
98,6aBUnkXuCEQQHAlTokv9or,This Must Be the Place (Naive Melody) - 2005 R...,Talking Heads,0.942,0.641,7,-10.150,1,0.0333,0.33100,0.179000,0.028,0.966,114.443,audio_features,spotify:track:6aBUnkXuCEQQHAlTokv9or,https://api.spotify.com/v1/tracks/6aBUnkXuCEQQ...,https://api.spotify.com/v1/audio-analysis/6aBU...,296147,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Jazz playlist

In [12]:
id_song_artist_df8 = get_id_song_artist("spotify", "37i9dQZF1DXbITWG1ZJKYt")
audio_features_df8 = get_playlist_audio_features("37i9dQZF1DXbITWG1ZJKYt")

merged_df8 = pd.merge(id_song_artist_df8, audio_features_df8, on='id', how='inner')
merged_df8.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df8.dropna(inplace=True)
merged_df8

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,29U7stRjqHU6rMiS8BfaI9,What A Wonderful World,Louis Armstrong,0.271,0.1650,5,-20.652,1,0.0351,0.729,0.000002,0.1180,0.2030,77.082,audio_features,spotify:track:29U7stRjqHU6rMiS8BfaI9,https://api.spotify.com/v1/tracks/29U7stRjqHU6...,https://api.spotify.com/v1/audio-analysis/29U7...,139227,4
1,6VTbbVjKOC2qWagIDbkJrC,My Baby Just Cares for Me - 2013 Remastered Ve...,Nina Simone,0.746,0.2430,4,-15.134,0,0.1230,0.829,0.000001,0.0905,0.4250,118.578,audio_features,spotify:track:6VTbbVjKOC2qWagIDbkJrC,https://api.spotify.com/v1/tracks/6VTbbVjKOC2q...,https://api.spotify.com/v1/audio-analysis/6VTb...,215747,4
2,3898C4AbdbptwYet6547e5,The Girl From Ipanema,Stan Getz,0.664,0.1940,5,-15.487,0,0.0357,0.909,0.000039,0.0829,0.4430,129.863,audio_features,spotify:track:3898C4AbdbptwYet6547e5,https://api.spotify.com/v1/tracks/3898C4Abdbpt...,https://api.spotify.com/v1/audio-analysis/3898...,321827,4
3,648TTtYB0bH0P8Hfy0FmkL,Unforgettable,Nat King Cole,0.349,0.1820,5,-13.507,1,0.0310,0.920,0.015200,0.1430,0.1780,136.094,audio_features,spotify:track:648TTtYB0bH0P8Hfy0FmkL,https://api.spotify.com/v1/tracks/648TTtYB0bH0...,https://api.spotify.com/v1/audio-analysis/648T...,191973,3
4,2IdwQxhJn9ZE4zIotcCefR,Stormy Weather,Etta James,0.262,0.3560,0,-8.031,0,0.0303,0.613,0.000000,0.1750,0.4260,174.176,audio_features,spotify:track:2IdwQxhJn9ZE4zIotcCefR,https://api.spotify.com/v1/tracks/2IdwQxhJn9ZE...,https://api.spotify.com/v1/audio-analysis/2Idw...,187667,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,34zhjduFdQFhkRUJ230h9T,April In Paris,Billie Holiday,0.496,0.1270,0,-14.211,1,0.0382,0.927,0.000071,0.1320,0.2110,74.746,audio_features,spotify:track:34zhjduFdQFhkRUJ230h9T,https://api.spotify.com/v1/tracks/34zhjduFdQFh...,https://api.spotify.com/v1/audio-analysis/34zh...,185587,4
245,1CTex49P0iWwzUGsMNjgaV,Strange Fruit,Billie Holiday,0.313,0.0437,6,-22.627,1,0.0315,0.989,0.148000,0.2090,0.0625,100.827,audio_features,spotify:track:1CTex49P0iWwzUGsMNjgaV,https://api.spotify.com/v1/tracks/1CTex49P0iWw...,https://api.spotify.com/v1/audio-analysis/1CTe...,192867,1
246,5DIVWgTeJ2fPIxaY9e7ZKn,I've Got A Crush On You,Ella Fitzgerald,0.455,0.1060,2,-14.086,1,0.0422,0.986,0.000000,0.1220,0.2580,66.807,audio_features,spotify:track:5DIVWgTeJ2fPIxaY9e7ZKn,https://api.spotify.com/v1/tracks/5DIVWgTeJ2fP...,https://api.spotify.com/v1/audio-analysis/5DIV...,196440,4
247,5BLr7EEgi2JdKbMbYjFXas,It's A Lovely Day Today,Ella Fitzgerald,0.573,0.2750,8,-10.684,1,0.0534,0.840,0.000000,0.1960,0.5880,140.900,audio_features,spotify:track:5BLr7EEgi2JdKbMbYjFXas,https://api.spotify.com/v1/tracks/5BLr7EEgi2Jd...,https://api.spotify.com/v1/audio-analysis/5BLr...,148267,4


#### Obtaining a collection of songs & artists, and audio features from a Spotify Classical playlist

In [13]:
id_song_artist_df9 = get_id_song_artist("spotify", "37i9dQZF1DWWEJlAGA9gs0")
audio_features_df9 = get_playlist_audio_features("37i9dQZF1DWWEJlAGA9gs0")

merged_df9 = pd.merge(id_song_artist_df9, audio_features_df9, on='id', how='inner')
merged_df9.drop_duplicates(subset=['name'], keep='first', inplace=True)
merged_df9.dropna(inplace=True)
merged_df9

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,61dYvvfIRtIDFuqZypPAta,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",Johann Sebastian Bach,0.2570,0.1090,7,-21.356,1,0.0597,0.900,0.867,0.0762,0.3070,75.020,audio_features,spotify:track:61dYvvfIRtIDFuqZypPAta,https://api.spotify.com/v1/tracks/61dYvvfIRtID...,https://api.spotify.com/v1/audio-analysis/61dY...,151480,4
1,4SFBV7SRNG2e2kyL1F6kjU,"The Well-Tempered Clavier, Book 1, BWV 846-869...",Johann Sebastian Bach,0.2210,0.1260,0,-25.427,1,0.0447,0.989,0.897,0.1020,0.2160,133.630,audio_features,spotify:track:4SFBV7SRNG2e2kyL1F6kjU,https://api.spotify.com/v1/tracks/4SFBV7SRNG2e...,https://api.spotify.com/v1/audio-analysis/4SFB...,139307,4
2,3zLTPuucd3e6TxZnu2dlVS,"Bagatelle No. 25 in A Minor, WoO 59 ""Für Elise""",Ludwig van Beethoven,0.2890,0.0306,9,-30.790,0,0.0446,0.987,0.911,0.1020,0.1180,125.610,audio_features,spotify:track:3zLTPuucd3e6TxZnu2dlVS,https://api.spotify.com/v1/tracks/3zLTPuucd3e6...,https://api.spotify.com/v1/audio-analysis/3zLT...,212067,3
3,0DqFXzLtKkj9JO421NJpl1,"Suite No. 3 in D Major, BWV 1068: II. Air",Johann Sebastian Bach,0.0769,0.0846,2,-25.997,1,0.0450,0.922,0.929,0.1130,0.0586,176.329,audio_features,spotify:track:0DqFXzLtKkj9JO421NJpl1,https://api.spotify.com/v1/tracks/0DqFXzLtKkj9...,https://api.spotify.com/v1/audio-analysis/0DqF...,363253,4
4,1Mse9NKBbEASi50CQ4aYhr,Pachelbel: Canon and Gigue for Three Violins a...,Johann Pachelbel,0.1280,0.1320,2,-17.670,1,0.0353,0.744,0.942,0.1070,0.0988,85.314,audio_features,spotify:track:1Mse9NKBbEASi50CQ4aYhr,https://api.spotify.com/v1/tracks/1Mse9NKBbEAS...,https://api.spotify.com/v1/audio-analysis/1Mse...,301587,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,4VVEfBXep7FdOt524jkfLF,"Young Birches, Op. 128, No. 2",Amy Beach,0.2400,0.0834,6,-29.545,0,0.0499,0.978,0.944,0.0804,0.1880,169.621,audio_features,spotify:track:4VVEfBXep7FdOt524jkfLF,https://api.spotify.com/v1/tracks/4VVEfBXep7Fd...,https://api.spotify.com/v1/audio-analysis/4VVE...,159320,4
146,7hOLwSaTSkrqxSMzj4bXI1,"Violin Concerto In D Major, Op. 35: III. Final...",Erich Wolfgang Korngold,0.4050,0.3180,9,-17.140,1,0.0565,0.938,0.723,0.2840,0.3420,151.434,audio_features,spotify:track:7hOLwSaTSkrqxSMzj4bXI1,https://api.spotify.com/v1/tracks/7hOLwSaTSkrq...,https://api.spotify.com/v1/audio-analysis/7hOL...,424653,4
147,76i4IBw3KsuDw783tvHB0e,"Tahiti Trot, Op. 16 (orchestral transcription ...",Dmitri Shostakovich,0.2910,0.0981,8,-23.156,1,0.0398,0.950,0.865,0.2750,0.1940,138.168,audio_features,spotify:track:76i4IBw3KsuDw783tvHB0e,https://api.spotify.com/v1/tracks/76i4IBw3KsuD...,https://api.spotify.com/v1/audio-analysis/76i4...,249640,4
148,53ann6bENp7BF8uklrzQAp,The Promise,Michael Nyman,0.0992,0.2260,9,-14.840,0,0.0339,0.937,0.903,0.1830,0.0605,79.518,audio_features,spotify:track:53ann6bENp7BF8uklrzQAp,https://api.spotify.com/v1/tracks/53ann6bENp7B...,https://api.spotify.com/v1/audio-analysis/53an...,256493,4


### Concatenating all previously obtained dataframes to the final dataframe

In [14]:
final_sp_df = pd.concat([merged_df1, merged_df2, merged_df3, merged_df4, merged_df5, merged_df6, merged_df7, merged_df8, merged_df9], axis=0, ignore_index=True)
final_sp_df.drop_duplicates(subset=['name'], keep='first', inplace=True)
final_sp_df

,id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,33xMbeHzmWd6Od0BmLZEUs,2K,Nosaj Thing,0.3100,0.4450,7,-13.355,0,0.0863,0.0940,0.06780,0.1130,0.1220,95.360,audio_features,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560,3
1,3UnyplmZaq547hwsfOR5yy,4 Billion Souls,The Doors,0.4190,0.5650,5,-11.565,1,0.0347,0.1370,0.33700,0.1280,0.6480,151.277,audio_features,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707,4
2,1w8QCSDH4QobcQeT4uMKLm,4 Minute Warning,Radiohead,0.3540,0.3020,9,-13.078,1,0.0326,0.5900,0.07090,0.1110,0.2230,123.753,audio_features,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285,4
3,7J9mBHG4J2eIfDAv5BehKA,7 Element,Vitas,0.7270,0.7850,5,-6.707,0,0.0603,0.3250,0.12600,0.3100,0.9600,129.649,audio_features,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940,4
4,1VZedwJj1gyi88WFRhfThb,#9 Dream,R.E.M.,0.5710,0.7240,0,-5.967,1,0.0260,0.0231,0.00311,0.0919,0.3850,116.755,audio_features,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11560,4VVEfBXep7FdOt524jkfLF,"Young Birches, Op. 128, No. 2",Amy Beach,0.2400,0.0834,6,-29.545,0,0.0499,0.9780,0.94400,0.0804,0.1880,169.621,audio_features,spotify:track:4VVEfBXep7FdOt524jkfLF,https://api.spotify.com/v1/tracks/4VVEfBXep7Fd...,https://api.spotify.com/v1/audio-analysis/4VVE...,159320,4
11561,7hOLwSaTSkrqxSMzj4bXI1,"Violin Concerto In D Major, Op. 35: III. Final...",Erich Wolfgang Korngold,0.4050,0.3180,9,-17.140,1,0.0565,0.9380,0.72300,0.2840,0.3420,151.434,audio_features,spotify:track:7hOLwSaTSkrqxSMzj4bXI1,https://api.spotify.com/v1/tracks/7hOLwSaTSkrq...,https://api.spotify.com/v1/audio-analysis/7hOL...,424653,4
11562,76i4IBw3KsuDw783tvHB0e,"Tahiti Trot, Op. 16 (orchestral transcription ...",Dmitri Shostakovich,0.2910,0.0981,8,-23.156,1,0.0398,0.9500,0.86500,0.2750,0.1940,138.168,audio_features,spotify:track:76i4IBw3KsuDw783tvHB0e,https://api.spotify.com/v1/tracks/76i4IBw3KsuD...,https://api.spotify.com/v1/audio-analysis/76i4...,249640,4
11563,53ann6bENp7BF8uklrzQAp,The Promise,Michael Nyman,0.0992,0.2260,9,-14.840,0,0.0339,0.9370,0.90300,0.1830,0.0605,79.518,audio_features,spotify:track:53ann6bENp7BF8uklrzQAp,https://api.spotify.com/v1/tracks/53ann6bENp7B...,https://api.spotify.com/v1/audio-analysis/53an...,256493,4
